In [4]:
import freqml as fm
from freqml import *
from binance.client import Client
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
secret = "3XUJnKgPdzfhJVWcTC0adU3CW9T9AOksNW1oTJ0xp0ELDdnD5mxCUj5IpxpZWY9v"
key = "a4JbLRzlMP9aeiiaQpYTZewvkYWdml3JmkpEZr7ndkKAvsqqFKzPFd98Pfhb1BBs"

In [6]:
client = Client(key, secret)

In [7]:
df = fm.load_dataset(client)

In [8]:
df.shape

(188243, 7)

In [9]:
df.bars.period

Timedelta('1 days 00:00:22.855000')

In [10]:
df_TB = df.bars.TB()

### discover promising points

In [11]:
%%time
promising_points = fm.CUSUM(df)

CPU times: user 3.15 s, sys: 529 µs, total: 3.15 s
Wall time: 3.15 s


In [12]:
promising_points.shape

(2298, 1)

In [13]:
promising_points.sample(3)

,datetime
93,2020-07-20 22:34:53.324000+03:00
1247,2020-07-21 10:44:56.433000+03:00
711,2020-07-21 10:35:14.419000+03:00


### label dataset using triple barrier 

In [17]:
%%time
df_labels = fm.labeling(df_TB, promising_points[:2000], up=2, down=2, lapse=3)

CPU times: user 30.6 s, sys: 12.1 ms, total: 30.6 s
Wall time: 30.6 s


In [18]:
df_labels.sample(5)

,start,end,size,side,duration,touch_time
317,234.15,234.83,0.68,0,00:03:12.530000,2020-07-20 22:46:22.494000+03:00
938,240.71,240.78,0.07,0,00:03:09.238000,2020-07-21 10:41:19.753000+03:00
1524,242.11,242.56,0.45,0,00:03:00.296000,2020-07-21 11:03:17.786000+03:00
104,235.77,235.32,0.45,0,00:03:04.177000,2020-07-20 22:38:00.901000+03:00
1462,241.63,241.55,0.08,0,00:03:09.371000,2020-07-21 10:54:01.762000+03:00


In [19]:
df_labels.shape

(2000, 6)

### meta-label dataset using triple barrier

In [20]:
predictions = pd.DataFrame(np.random.choice([-1, 1], df_labels.shape[0]))

In [23]:
%%time
df_meta_labels = fm.meta_labeling(df_TB, promising_points[:2000], predictions)

CPU times: user 29.3 s, sys: 20.8 ms, total: 29.3 s
Wall time: 29.3 s


In [24]:
df_meta_labels.sample(5)

,start,end,size,pred_side,true_side,take,duration,touch_time
1857,244.54,244.50,0.04,1,0,0,00:05:16.830000,2020-07-21 16:08:46.137000+03:00
1684,242.34,242.12,0.22,-1,0,0,00:05:21.979000,2020-07-21 12:48:09.892000+03:00
1406,241.73,241.36,0.37,-1,0,0,00:05:10.155000,2020-07-21 10:52:14.474000+03:00
1954,243.99,243.77,0.22,1,0,0,00:05:15.045000,2020-07-21 16:36:51.089000+03:00
1603,241.72,241.95,0.23,1,0,0,00:05:12.570000,2020-07-21 11:19:33.538000+03:00


In [25]:
df_meta_labels.shape

(2000, 8)